In [ ]:
#libraries
from pathlib import Path

import pandas as pd
import math
import logging
import datetime as dt
import os 
import re
from tqdm import tqdm
import glob
!pip install utils
import utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 58k bert result

In [ ]:
from google.colab import drive
drive.mount('/content/mnt')
%cd /content/mnt/MyDrive/

#path to the file/output of model
path_in = Path.cwd() / "data/in/ai_phase/incident_inference_220314.xlsx"
#path_in = Path.cwd() / "../data/in/ai_phase/incident_inference_220314.xlsx"
df_58k = pd.read_excel(path_in, index_col=0)

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).
/content/mnt/MyDrive


In [ ]:
df_58k = df_58k.reset_index()
df_58k.head()
df_58k.describe()

,id,report,tokenized_report,IOB_report,outs_pos,outs_int,outs_rel
count,58707,58707,58707,58707,58707,58707,58707
unique,58707,58656,58656,57796,57814,58129,57612
top,A0256111ECEC35CC1,重複投与,"['▁', '重複', '投与']","['O', 'B-Drug', 'O']","['O', 'B-Drug', 'O']","['IA', 'IA', 'O']","['O', '1', '1', '1', 'O', 'O']"
freq,1,9,9,19,20,18,18


In [ ]:
#convert list input to string in dataframe
df_58k.outs_pos = df_58k.outs_pos.apply(eval)
df_58k.outs_int = df_58k.outs_int.apply(eval)
df_58k.outs_rel = df_58k.outs_rel.apply(eval)
df_58k.tokenized_report = df_58k.tokenized_report.apply(eval)

In [ ]:
df_total = pd.DataFrame()
for idx, row in tqdm(df_58k.loc[:].iterrows()):

    ans = []
    for idx2, ele in enumerate(row.outs_pos):
        
        mini = []
        if ele.startswith("B"):
            count = idx2
            while count <= len(row.outs_pos)-2:
                mini.append(count)
                count +=1
                if row.outs_pos[count] == "O" or row.outs_pos[count].startswith("B"):
                    break
        if mini != []:
            ans.append(mini)
    df_sub = pd.DataFrame(columns=["id","reports","entity_name","start_idx","end_idx","entity", "label","rel_index"])        
    final_ans = []
    for a in ans:
        entity = "".join(row.tokenized_report[a[0]:a[-1]+1])
                                   
        ent_e = row.outs_pos[a[0]:a[-1]+1]
        ent_e = [ele.replace("I-","") for ele in ent_e]
        ent_e = [ele.replace("B-","") for ele in ent_e]
        ent_e = list(set(ent_e))
    
        for ele in reversed(ent_e):
            if ele in ["O","<pad>"]:
                ent_e.remove(ele)
        if len(ent_e) > 0:
            ent_ele = ent_e[0]
        else:
            ent_ele = None
                                   
        int_f = row.outs_int[a[0]:a[-1]+1]
        int_f = list(set(int_f))
        for ele in reversed(int_f):
            if ele in ["O","<pad>"]:
                int_f.remove(ele)
        if len(int_f) > 0:
            int_fac_ele = int_f[0]
        else:
            int_fac_ele = None
        
        rel_a = row.outs_rel[a[0]:a[-1]+1]
        
        rel_a = list(set(rel_a))
        for ele in reversed(rel_a):
            if ele in ["O","<pad>"]:
                rel_a.remove(ele)
        if len(rel_a) > 0:
            rel_a_ele = rel_a[0]
        else:
            rel_a_ele = None
        if len(row.tokenized_report) > 1:
            test = "".join(list(row.tokenized_report[0].replace("▁","")) + list(row.tokenized_report[1:a[0]]))
        else:
            print("error_occured")
            
        start_idx = len(test)+1
        end_idx = start_idx + len(entity)
        new_row = [row.id, row.report,entity, start_idx, end_idx, ent_ele, int_fac_ele, rel_a_ele]
        df_sub.loc[df_sub.shape[0]] = new_row
    df_total = pd.concat([df_total, df_sub])
df_total  

58707it [1:06:44, 14.66it/s]


,id,reports,entity_name,start_idx,end_idx,entity,label,rel_index
0,A0256111ECEC35CC1,気管支喘息発作の治療としてメプチン・べネトリンの吸入を行ったが効果なくソルメドロール125m...,メプチン・,14,19,Drug,IA,1
1,A0256111ECEC35CC1,気管支喘息発作の治療としてメプチン・べネトリンの吸入を行ったが効果なくソルメドロール125m...,吸入,25,27,Route,IA,1
2,A0256111ECEC35CC1,気管支喘息発作の治療としてメプチン・べネトリンの吸入を行ったが効果なくソルメドロール125m...,ソルメドロール,36,43,Drug,IA,1
3,A0256111ECEC35CC1,気管支喘息発作の治療としてメプチン・べネトリンの吸入を行ったが効果なくソルメドロール125m...,125mg,43,48,Strength_amount,IN,1
4,A0256111ECEC35CC1,気管支喘息発作の治療としてメプチン・べネトリンの吸入を行ったが効果なくソルメドロール125m...,生理食塩水,49,54,Drug,IA,1
...,...,...,...,...,...,...,...,...
1,AFD8FEDAB0A89ED9A,甲状腺癌、術後嘔気でプリンペラン・ナウゼリンを投与、不穏でセレネースを1a使用。約半日でプリ...,ナウゼリン,18,23,Drug,IA,1
2,AFD8FEDAB0A89ED9A,甲状腺癌、術後嘔気でプリンペラン・ナウゼリンを投与、不穏でセレネースを1a使用。約半日でプリ...,セレネース,30,35,Drug,IA,1
3,AFD8FEDAB0A89ED9A,甲状腺癌、術後嘔気でプリンペラン・ナウゼリンを投与、不穏でセレネースを1a使用。約半日でプリ...,1a,36,38,Dosage,NA,1
4,AFD8FEDAB0A89ED9A,甲状腺癌、術後嘔気でプリンペラン・ナウゼリンを投与、不穏でセレネースを1a使用。約半日でプリ...,プリンペラン,45,51,Drug,IA,1


In [ ]:
SAVE_PATH = "AI_outcomes.xlsx"
df_total.to_excel(SAVE_PATH, index=False)